In [1]:
from collections import defaultdict
from aocutils.grid import iterate, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, dimensions, manhattan, positive, conv1d, conv2d
from aocutils.maze import bfs, dijkstra, get_path, dfs
import heapq

In [5]:
lines = open('p2.txt').read().splitlines()
lines = [list(line) for line in lines]
neigh = arr_neighbors(lines)
maze = grid_to_dict(lines)
conn = {}

def get_taken(maze):
    taken = [[],[],[],[]]
    locs = set()
    for k,v in maze.items():
        if v != ' ' and v != '#':
            conn[k] = {n for n in neigh[k] if maze[n]!=' ' and maze[n] != '#'}
            locs.add(k)
        
            match v:
                case 'A': taken[0].append(k)
                case 'B': taken[1].append(k)
                case 'C': taken[2].append(k)
                case 'D': taken[3].append(k)
    return locs, tuple(l for num in taken for l in num)

# now compute the paths between all the points
def get_paths(locs):
    paths = {}
    for start in locs:
        for end in locs:
            if start != end:
                # store the locations between start and end
                paths[start, end] = set(bfs(conn, start, end)[1:-1])
    return paths

def goal_reached_fn(locs):
    if [l[1] for l in locs] == [3,3,3,3,5,5,5,5,7,7,7,7,9,9,9,9]:
        return True
    return False
assert goal_reached_fn([(3,3),(3,3),(3,3),(3,3),(5,5),(5,5),(5,5),(5,5),(7,7),(7,7),(7,7),(1,7),(9,9),(9,9),(9,9),(1,9)]) == True

In [6]:

def getneighbors(state):    
    # compute the goals for each column, there is a goal if there are no wrong Amphipods in column and there is space
    # in that case the goal is the bottom empty square
    def get_goals(free, taken):
        nextgoal = defaultdict(int)
        for col in (3,5,7,9):
            for row in (5,4,3,2):
                if (row,col) in free:
                    nextgoal[col] = row, col
                    break
                rowindex = taken.index((row,col))
                if targets[rowindex] != col:
                    break
        return nextgoal
    
    def get_state(amphi, goal):
        path = paths[amphi, goal]
        if path == path & free:
            newcost = costs[idx] * (len(path) + 1) + cost
            newlocs = taken.copy()
            newlocs[idx] = goal
            return newcost, tuple(newlocs)
        else:
            return False

    options = []
    cost, taken = state
    free = locs - set(taken)
    taken = list(taken)
    nextgoal = get_goals(free, taken)
                
    for idx, amphi in enumerate(taken):
        # go over the amphipods and see if and where they can move
        row, col  = amphi
        targetcol = targets[idx]
        goal = nextgoal[targetcol]
        if goal and col == targetcol and row > goal[0]:
            # already at location
            continue
        else:
            if goal:
                opt = get_state(amphi, goal)
                # if there is a goal with a free path, move there immediately, no regret
                if opt: return [opt]
                
            #no goal atm or path to goal is blocked
            if row != 1:
                # we are not at first row and target is not same col (so in wrong column)
                for f in firstrow & free:
                    if f[0] == 1 and not f[1] in (3,5,7,9):
                        # we want to move to first row
                        opt = get_state(amphi, f)
                        if opt:
                            # the way is free
                            options.append(opt)
    return options

costs = {0:1, 1:1, 2:1,3:1, 4:10, 5:10, 6:10, 7:10, 8:100, 9:100,10:100,11:100,12:1000,13:1000,14:1000, 15:1000}
targets = {0:3, 1:3, 2:3,3:3, 4:5, 5:5, 6:5, 7:5, 8:7, 9:7,10:7,11:7,12:9,13:9,14:9, 15:9}
locs, taken = get_taken(maze)
firstrow = {l for l in locs if l[1] not in (3,5,7,9) and l[0] == 1}
start = 0, taken
paths = get_paths(locs)
assert len(getneighbors(start)) == 28


In [7]:

def dijkstra(connections,start, goal=None):
    """
    Requires a dict with as values a LIST of tuples (neighbor, weight)
    Or a function returning a list of tuples with neighbors and weights per node

    Returns
    if goal == None:    return all paths from start
    elif goal found:    returns path to goal
    else:               returns False
    """
    
    frontier = [start] # the locations that still need to be visited
    parents = {start[0]: (None,0)}

    while frontier:
        new = heapq.heappop(frontier)
        search_cost, search_node = new
        if goal(search_node): 
            print('goal found')
            return search_cost, search_node, parents
        neighbors = connections(new)
        if neighbors:
            for n in neighbors:
                newcost, newlocs = n
                if newlocs not in parents:
                    if newlocs not in parents or newcost < parents[newlocs][1]:
                        heapq.heappush(frontier,(newcost, newlocs))
                        parents[newlocs]= search_node,newcost
    else: return False
search_cost, search_node, parents = dijkstra(getneighbors, start, goal_reached_fn)
search_cost


goal found


50132